Install relevant modules and packages

In [19]:
!apt-get install -y dcm2niix # '!' runs commands in the console
!apt-get install -y parallel

# dependencies for freesurfer
!apt-get install -y wget
!apt-get install -y grep
!apt-get install -y tcsh
!apt-get install -y bc

!pip install nibabel

import os
import glob
import nibabel as nib

# install freesurfer (might take a few minutes)
!git clone https://github.com/freesurfer/freesurfer.git
os.system('/content/freesurfer/install_freesurfer.sh')

# set the relevant freesurfer directories
os.environ['FREESURFER_HOME'] = '/content/freesurfer'
os.environ['SUBJECTS_DIR'] = '/content/freesurfer_output'
os.environ['FS_LICENSE'] = '/content/freesurfer_license.txt'
os.environ['PATH'] += ':/content/freesurfer/bin'
os.system('source /content/freesurfer/SetUpFreeSurfer.sh')

!source /content/freesurfer/SetUpFreeSurfer.sh

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
dcm2niix is already the newest version (1.0.20211006-1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
parallel is already the newest version (20210822+ds-2).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
wget is already the newest version (1.21.2-2ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
grep is already the newest version (3.7-1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tcsh is already the newest version

Get anonymized (T1 weighted) dicom files for analysis, and create temporary folders to store these and other files.

In [21]:
# clone the repository to get the raw dicoms
!git clone https://github.com/datalad/example-dicom-structural

# create folders for the NIfTI conversions and corresponding recons
!mkdir /content/nii_files/
!mkdir /content/freesurfer_output/

'''
Note: "T1 weighting" refers to a form of MRI image processing performed by technicians.
Online resources will nearly always provide processed MRI images,
though they may not be T1 weighted
''';

fatal: destination path 'example-dicom-structural' already exists and is not an empty directory.
mkdir: cannot create directory ‘/content/nii_files/’: File exists
mkdir: cannot create directory ‘/content/freesurfer_output/’: File exists


Convert the DICOMs to NIfTIs

In [22]:
# prepare paths
INPUT_PATH = '/content/example-dicom-structural/dicoms/'
OUTPUT_PATH = '/content/nii_files/'

# grep to ignore some warnings regarding the manufacturer (since we're using sample dicoms)
!dcm2niix -o '{OUTPUT_PATH}' '{INPUT_PATH}' | grep -v "Unknown manufacturer"

Chris Rorden's dcm2niiX version v1.0.20211006  (JP2:OpenJPEG) GCC11.2.0 x86-64 (64-bit Linux)
Found 384 DICOM file(s)
Convert 384 DICOM as /content/nii_files/dicoms_anat-T1w_20130717141500_401d (274x384x384x1)
Conversion required 0.576597 seconds (0.570945 for core code).


Reconstruct the NIfTI files.

Parallel allows you to recon multiple subjects simultaneously by recruiting n CPU cores (determined by --jobs n)

In [65]:
# prepare paths
INPUT_PATH = '/content/freesurfer_output'
nii_paths = glob.glob('/content/nii_files/*.nii')
ALL_NII = ' '.join(nii_paths)  # format that parallel wants

# use parallel to execute recon-all on each NIfTI file
!parallel --jobs 2 recon-all -i {} -s {/.} -all ::: /content/nii_files/dicoms_anat-T1w_20130717141500_401a.nii

fs-check-version --s dicoms_anat-T1w_20130717141500_401a --o /tmp/tmp.WxDNIs
Fri Oct 25 10:49:23 PM UTC 2024

setenv SUBJECTS_DIR /content/freesurfer_output
cd /content
/content/freesurfer/bin/fs-check-version --s dicoms_anat-T1w_20130717141500_401a --o /tmp/tmp.WxDNIs
-rwxrwxr-x 1 1623927 1623927 18565 Jun 14  2023 /content/freesurfer/bin/fs-check-version

freesurfer-linux-ubuntu22_x86_64-7.4.1-20230614-7eb8460
$Id$
Linux 07abd3fb8efa 6.1.85+ #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024 x86_64 x86_64 x86_64 GNU/Linux
pid 25102
Current FS Version freesurfer-linux-ubuntu22_x86_64-7.4.1-20230614-7eb8460
Subject does not have a bstampfile, copying /content/freesurfer/build-stamp.txt
Subject FS Version: freesurfer-linux-ubuntu22_x86_64-7.4.1-20230614-7eb8460
No constraints on version because REQ=UnSet and FsVerFile=NotThere
#@#% fs-check-version match = 1
fs-check-version Done
INFO: SUBJECTS_DIR is /content/freesurfer_output
Actual FREESURFER_HOME /content/freesurfer
Linux 07abd3fb8

In [58]:
INPUT_PATH = '/content/freesurfer_output'
nii_paths = glob.glob('/content/nii_files/*.nii')
ALL_NII = ' '.join(nii_paths)  # format that parallel wants

'/content/nii_files/dicoms_anat-T1w_20130717141500_401a.nii /content/nii_files/dicoms_anat-T1w_20130717141500_401c.nii /content/nii_files/dicoms_anat-T1w_20130717141500_401b.nii /content/nii_files/dicoms_anat-T1w_20130717141500_401d.nii /content/nii_files/dicoms_anat-T1w_20130717141500_401.nii'